# 🧪 Simple Ollama Environment — Google Colab (Fixed)

This notebook recreates the **Simple Ollama Environment** (Python 3 + Jupyter + Ollama) in a Google Colab runtime.

It will:

1. Install the Python client **`ollama`**.
2. Download the **Ollama server** binary (Linux, no root required).
3. Start the Ollama server in the background inside this notebook.
4. Pull a small model (default: `qwen2.5:0.5b-instruct`).
5. Run a minimal chat example via the Python client.

> ⚠️ **Note:** Colab runtimes are ephemeral. If the runtime restarts, you need to rerun the setup cells.


## 0. (Optional) Enable GPU in Colab

Before running the setup cells, you can enable a GPU:

1. In the Colab menu, go to **Runtime → Change runtime type**.
2. Set **Hardware accelerator** to **GPU**.
3. Click **Save**.

Small models will also run on CPU, just more slowly.


In [ ]:
import subprocess

print("🔎 Checking for NVIDIA GPU...")
try:
    gpu_info = subprocess.check_output(["nvidia-smi"], stderr=subprocess.STDOUT, text=True)
    print("✅ GPU detected! Summary:\n")
    # Show only first few lines
    print("\n".join(gpu_info.splitlines()[:12]))
except Exception:
    print("⚠️ No NVIDIA GPU detected or driver unavailable.")
    print("   Colab CPU-only runtimes can still run small models, just a bit slower.")


## 1. Install Python dependencies

We install the official **Ollama Python library** plus `requests` (for simple health checks).


In [ ]:
!pip install -q "ollama>=0.6,<1.0" requests

import importlib
spec = importlib.util.find_spec("ollama")
print("✅ Installed 'ollama' Python client." if spec is not None else "❌ Failed to install 'ollama' client.")


## 2. Download the Ollama server binary (no root)

We fetch the pre-built **Linux x86_64** Ollama binary directly into your home directory (no `sudo`, no systemd changes).

This follows the **non-root** installation pattern suggested in the Ollama community docs.


In [ ]:
import os
import pathlib
import subprocess

HOME = os.path.expanduser("~")
OLLAMA_HOME = os.path.join(HOME, "opt", "ollama")
SRC_DIR = os.path.join(HOME, "src")

os.makedirs(OLLAMA_HOME, exist_ok=True)
os.makedirs(SRC_DIR, exist_ok=True)

tar_path = os.path.join(SRC_DIR, "ollama-linux-amd64.tgz")
# Corrected path: the ollama binary is extracted into the 'bin' subdirectory
ollama_bin_path = os.path.join(OLLAMA_HOME, "bin", "ollama")

if not os.path.exists(ollama_bin_path):
    print("⬇️ Downloading Ollama Linux binary...")
    subprocess.run(
        [
            "curl",
            "-L",
            "https://ollama.com/download/ollama-linux-amd64.tgz",
            "-o",
            tar_path,
        ],
        check=True,
    )

    # Verify downloaded file size. Ollama binary is usually several MBs.
    if not os.path.exists(tar_path) or os.path.getsize(tar_path) < 1024 * 1024: # Less than 1MB is suspicious
        raise RuntimeError(f"Downloaded Ollama tarball is missing or too small: {tar_path}")

    print("📦 Extracting...")
    # The tar command extracts contents directly into OLLAMA_HOME, but the binary is in a 'bin' subdir
    subprocess.run(
        ["tar", "-C", OLLAMA_HOME, "-xzf", tar_path],
        check=True,
    )

    # Verify existence immediately after extraction with the corrected path
    if not os.path.exists(ollama_bin_path):
        print(f"⚠️ Ollama binary not found after extraction. Listing contents of {OLLAMA_HOME} recursively for diagnosis:")
        ls_output = subprocess.check_output(["ls", "-laR", OLLAMA_HOME], text=True, stderr=subprocess.STDOUT)
        print(ls_output) # Print the captured output
        raise RuntimeError(f"Ollama binary not found at {ollama_bin_path} after extraction. See directory listing above.")
else:
    print("✅ Ollama binary already present, skipping download.")

# Final check (redundant if previous checks are robust, but good for safety)
if not os.path.exists(ollama_bin_path):
    raise RuntimeError(f"Ollama binary not found at {ollama_bin_path}")

print("✅ Ollama binary ready at:", ollama_bin_path)

## 3. Start the Ollama server in the background

This cell:

- Configures some basic environment variables.
- Starts `ollama serve` as a **background process** inside the Python kernel.
- Waits until the HTTP API is healthy at `http://127.0.0.1:11434`.


In [ ]:
import os
import time
import subprocess
import requests

HOME = os.path.expanduser("~")
OLLAMA_HOME = os.path.join(HOME, "opt", "ollama")
# Corrected OLLAMA_BIN path to point to the 'bin' subdirectory
OLLAMA_BIN = os.path.join(OLLAMA_HOME, "bin", "ollama")

if not os.path.exists(OLLAMA_BIN):
    raise RuntimeError("Ollama binary not found. Run the previous install cell first.")

# Stop an existing server (if any) from a previous run
try:
    OLLAMA_SERVER_PROCESS  # type: ignore[name-defined]
    if OLLAMA_SERVER_PROCESS.poll() is None:
        print("🛑 Stopping existing Ollama server...")
        OLLAMA_SERVER_PROCESS.terminate()
        try:
            OLLAMA_SERVER_PROCESS.wait(timeout=10)
        except subprocess.TimeoutExpired:
            OLLAMA_SERVER_PROCESS.kill()
except NameError:
    pass

# Where to store models (in the Colab filesystem)
os.environ["OLLAMA_MODELS"] = os.path.join("/content", "ollama-models")
os.makedirs(os.environ["OLLAMA_MODELS"], exist_ok=True)

# Basic tuning flags
os.environ["OLLAMA_MAX_LOADED_MODELS"] = "1"
os.environ["OLLAMA_NUM_PARALLEL"] = "1"

# Host where the API will be available
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

print("🚀 Starting Ollama server...")
print(f"   Models directory: {os.environ['OLLAMA_MODELS']}")
print(f"   Server host: {os.environ['OLLAMA_HOST']}")

OLLAMA_SERVER_PROCESS = subprocess.Popen(
    [OLLAMA_BIN, "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
    env=os.environ.copy()
)

# Wait for health endpoint
print("⏳ Waiting for server to start...")
for i in range(60):
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=2)
        if r.ok:
            print("✅ Ollama server is up at http://127.0.0.1:11434")
            break
    except Exception:
        pass
    time.sleep(1)
else:
    raise RuntimeError("Ollama server did not become healthy in time.")

## 4. Pull a small model

We'll pull the same small instruction-tuned model used in the original **Simple Ollama Environment**:

- Default: `qwen2.5:0.5b-instruct`

You can change the model name before running the cell (e.g. to `llama3.2:1b` or `qwen2:0.5b`, if available on Ollama).


In [ ]:
import os
import subprocess

# Change this if you want a different model
MODEL_NAME = "qwen2.5:0.5b-instruct"

HOME = os.path.expanduser("~")
OLLAMA_HOME = os.path.join(HOME, "opt", "ollama")
# Corrected OLLAMA_BIN path to point to the 'bin' subdirectory
OLLAMA_BIN = os.path.join(OLLAMA_HOME, "bin", "ollama")

# Ensure environment variables are set
if "OLLAMA_HOST" not in os.environ:
    os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

print(f"📥 Pulling model: {MODEL_NAME}")
print(f"   Using OLLAMA_HOST: {os.environ['OLLAMA_HOST']}")

# Run with captured output for better error diagnosis
result = subprocess.run(
    [OLLAMA_BIN, "pull", MODEL_NAME],
    env=os.environ.copy(),
    capture_output=True,
    text=True
)

# Show output
if result.stdout:
    print(result.stdout)

if result.returncode != 0:
    print("❌ Error pulling model:")
    print(result.stderr)
    raise RuntimeError(f"Failed to pull model {MODEL_NAME}. See error above.")

print("✅ Model pulled and ready.")

## 4.5 Verify Model Installation

Let's verify the model was successfully pulled and is available.


In [ ]:
import os
import subprocess

HOME = os.path.expanduser("~")
OLLAMA_HOME = os.path.join(HOME, "opt", "ollama")
OLLAMA_BIN = os.path.join(OLLAMA_HOME, "bin", "ollama")

# Ensure environment variables are set
if "OLLAMA_HOST" not in os.environ:
    os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

print("📋 Listing available models:")
result = subprocess.run(
    [OLLAMA_BIN, "list"],
    env=os.environ.copy(),
    capture_output=True,
    text=True
)

print(result.stdout)

if result.returncode != 0:
    print("❌ Error listing models:")
    print(result.stderr)

## 5. Run a quick chat from Python

Now that:

- the **Ollama server** is running, and  
- a model is pulled,

we can use the `ollama` Python client to send a simple chat request.


In [ ]:
import ollama
import os

# Ensure the client talks to our local server
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

MODEL_NAME = "qwen2.5:0.5b-instruct"  # keep in sync with the pull cell above

print("✅ Using Ollama Python client version:", getattr(ollama, "__version__", "unknown"))
print("🧠 Talking to model:", MODEL_NAME)
print(f"   Server: {os.environ['OLLAMA_HOST']}\n")

prompt = "Di' solo 'Ciao!' in italiano e poi dammi 1 consiglio per studiare meglio."

print("💬 User:")
print(prompt)
print("\n🤖 Generating response...\n")

try:
    resp = ollama.chat(
        model=MODEL_NAME,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
    
    print("🤖 Model reply:")
    print(resp["message"]["content"])
except Exception as e:
    print(f"❌ Error during chat: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure the server is running (check cell 3)")
    print("2. Verify the model was pulled successfully (check cell 4)")
    print(f"3. Check that OLLAMA_HOST is set: {os.environ.get('OLLAMA_HOST', 'NOT SET')}")

## 5.5 Streaming Chat Example

Here's an example of streaming responses for real-time token generation.


In [ ]:
import ollama
import os

# Ensure the client talks to our local server
os.environ["OLLAMA_HOST"] = "http://127.0.0.1:11434"

MODEL_NAME = "qwen2.5:0.5b-instruct"

prompt = "Write a short poem about AI in 4 lines."

print("💬 User:", prompt)
print("\n🤖 Model reply (streaming):\n")

try:
    stream = ollama.chat(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print()  # New line at the end
except Exception as e:
    print(f"❌ Error during streaming chat: {e}")

## 6. Next steps

You now have:

- An **Ollama server** running inside Colab.
- A **Python client** ready to call `ollama.chat` / `ollama.generate`.
- At least one **local model** pulled.

From here you can:

- Swap models (edit `MODEL_NAME` and re-run the pull + chat cells).
- Use `stream=True` in `ollama.chat` for streaming responses.
- Integrate Ollama with other libraries (LangChain, LlamaIndex, etc.) in additional cells.
- Try other models like:
  - `llama3.2:1b` or `llama3.2:3b`
  - `phi4:3.8b`
  - `gemma2:2b`
  - `deepseek-r1:1.5b`

Happy hacking! 🚀


## 7. Troubleshooting

If you encounter issues:

### Server not starting
- Check if port 11434 is already in use
- Re-run cell 3 to restart the server

### Model pull fails
- Verify internet connectivity
- Try a smaller model first (e.g., `tinyllama:1.1b`)
- Check Ollama library is available at https://ollama.com/library

### Out of memory
- Use smaller models on CPU
- With GPU, check memory usage with `!nvidia-smi`
- Reduce `OLLAMA_MAX_LOADED_MODELS` or `OLLAMA_NUM_PARALLEL`

### Runtime disconnected
- You'll need to re-run all cells from the beginning
- Models persist in `/content/ollama-models` but server needs restart
